In [ ]:
import nltk
nltk.download() 

In [ ]:
file = open('corpus.txt',errors = 'ignore')
content = file.read()
file.close()

In [ ]:
list_replace = ['\n', '.', '\'', ',', '(', ')', '-','+',':','<','>','/','"','"']
for i in list_replace:
    content = content.replace(i, ' ') #Replace special symbols in the file with spaces

In [ ]:
content = content.lower() #Require all words to be lowercase
wordlist = content.split()
worddict = set(wordlist) #Build my own dictionary

In [ ]:
open_stopwords = open('cleanup_words.txt',errors = 'ignore') #Import the stop word list 
stopwords = open_stopwords.readlines()
open_stopwords.close()
stopwords[:8] #Observe the stop word format

In [ ]:
for i in range(len(stopwords)):
    stopwords[i] = stopwords[i].strip() 
stopwords[:8] #Observe the stop word format

In [ ]:
for i in stopwords:
    worddict.discard(i) #Remove stop words from the dictionary

In [ ]:
worddict_list = list(worddict)

In [ ]:
import csv
import datetime
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gensim
from snownlp import seg
from snownlp import SnowNLP
from snownlp import sentiment
from gensim import corpora, models
from gensim.models import CoherenceModel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import warnings
warnings.filterwarnings("ignore")
import sys
maxInt = sys.maxsize
decrement = True
while decrement:
    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True
worddict_list = list(worddict), header = 0, index_col = False, engine='python',encoding = 'utf-8')
csv_data = comment[[(len(str(x)) > 100) for x in comment]]
print(csv_data.shape)

# 构造corpus
train = []
for i in range(csv_data.shape[0]):
    comment = csv_data.iloc[i,7].split()
    train.append(comment)
    
id2word = corpora.Dictionary(train)
corpus = [ id2word.doc2bow(sentence) for sentence in train]

# 一致性和困惑度计算
coherence_values = []
perplexity_values = []
model_list = []

for topic in range(15):
    lda_model = gensim.models.LdaMulticore(corpus = corpus, num_topics=topic+1, id2word = id2word, random_state=100, chunksize=100, passes=10, per_word_topics=True)
    perplexity = pow(2,-lda_model.log_perplexity(corpus)) 
    print(perplexity,end='   ')
    perplexity_values.append(round(perplexity,3))
    
    model_list.append(lda_model)
    coherencemodel = CoherenceModel(model=lda_model, texts=train, dictionary=id2word, coherence='c_v')
    coherence_values.append(round(coherencemodel.get_coherence(),3))

In [ ]:
x = range(1,21)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()